<a href="https://colab.research.google.com/github/wahyunh10/Car-Auction-Prices-Project/blob/main/Car_Auction_Prices_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Auction Prices Project**

**In this project, there are 4 steps that I do:**

1. Data Preparation (Data Cleaning)
  * Extract Raw Data
  * Fill and Drop Null Values
  * Standardization Values
  * Load Clean Data to CSV and Local MySQL Database
2. Exploration Data Analysis
  * Find Insight using Matplotlib and Seaborn
  * Outlier Analysis
  * Find Best and Worst Seller using Quantile Analysis
3. Machine Learning Modelling
  * Prediction Prices Cars using Regression
  * Clustering Quality of Cars using KMeans Clustering
4. Hypothesis Testing using MannWhitneyU

The dataset contains historical car auction sales prices, scraped from the outside internet sources collected in 2015. This dataset taken from kaggle (https://www.kaggle.com/tunguz/used-car-auction-prices).

This dataset contains 491641 rows and 15 columns:

* year : information of year production of the car
* make : brand of the car
* model : model of the brand car
* trim : spesific model of the car
* body : body type of the car
* transmission : transmission type of the car
* vin : unique id of the car
* state : code state where the transaction happened
* condition : rating condition of the car
* odometer : total distance of the car
* color : body color of the car
* interior : interior color of the car
* seller : place that sold the car
* mmr : market price of the car
* sellingprice : price of the car
* saledate : date when the car sold

# **Data Preparation**

In [1]:
# Import needed libraries
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

**Import raw data from csv**

In [ ]:
df_source = pd.read_csv(".csv",sep='\t')
df_source.head()

In [ ]:
# Check length of every row in dataset
idx = []
for i in range (df_source.shape[0]):
    l = len(df_source.iloc[i][0].split(','))
    idx.append(l)
    
# Give temporary name columns 
tempcol = []
for i in range((max(idx))):
    tempcol.append(i)

df_csv = pd.read_csv("car_prices.csv",names=tempcol) 

# Extract first row value to make it into columns name
col = []
tempcol = ['temp1','temp2']
for i in df_csv.iloc[0].values:
    if str(i) != 'nan':
        col.append(i)
col.extend(tempcol) 

df = df_csv.drop(0,axis=0)
df.columns=col
df.reset_index(drop=True,inplace=True)
df.head()

**Check data after columns problem solved**

In [ ]:
def data_info():
    listkolom=[]
    for i in (df.columns):
        listkolom.append(i)

    listtipe=[]
    for i in (df.columns):
        listtipe.append(df[i].dtypes)

    nullmany = df.isnull().sum().values

    nullpctx = []
    for i in nullmany:
        nullpctx.append(round(i/len(df),4)*100)

    unik = df.nunique().values

    samp = []
    for i in df.columns:
        samp.append(df[i].sample().values)

    return pd.DataFrame({
        'dataFeatures' : listkolom,
        'dataType' : listtipe,
        'null' : nullmany,
        'nullpct' : nullpctx,
        'unique' : unik,
        'SampleValue' : samp
    })

data_info()



> It looks that data successfully stored into dataframe, and only have 21 anomaly rows that stored in temp1 column. Column temp2 is empty and will be deleted later.



**Process to tackle columns overlapping problem**

In [ ]:
# Function to make all value has lowercase word
def LowerVal(col):
    newval = []
    for i in df[col].values:
        if str(i)=='nan':
            newval.append(np.nan)
        else:
            cap = str(i).lower()
            newval.append(cap)
    df[col]=newval

In [ ]:
# Apply to all objects columns
col_to_clean = df.select_dtypes(include='object').columns
for i in col_to_clean:
    LowerVal(i)

In [ ]:
# Fix some value in 'make' columns
makeFix = []
for i in range(df.shape[0]):
    if str(df['make'][i]) == 'nan':
        makeFix.append(np.nan)
    else:
        newMake = df['make'][i].split()[0]
        makeFix.append(newMake)

df['make'] = makeFix

df.loc[df[df['make']=='Mercedes-b'].index,'make'] = 'Mercedes-benz'
df.loc[df[df['make']=='Mercedes'].index,'make'] = 'Mercedes-benz'
df.loc[df[df['make']=='Land'].index,'make'] = 'Landrover'
df.loc[df[df['make']=='Chev'].index,'make'] = 'Chevrolet'

In [ ]:
df['transmission'].value_counts()

> From transmission column value, we can take conclusion rows that have 'sedan' as it value is the anomaly rows.

In [ ]:
# Take rows that have 'sedan' as it value
enum=list(enumerate(df['transmission']=='sedan'))
idx_inv = []
for idx,val in enum:
    if val == True:
        idx_inv.append(idx)
        
inv_data = df.iloc[idx_inv]

# drop body column in invalid dataframe
inv_data.drop(['body'],1,inplace=True)
inv_data.head()

> body column in invalid dataframe dropped because it contain wrong information

In [ ]:
df.drop(['temp2'],1,inplace=True)
inv_data.columns=df.columns

# Re-fill value for invalid rows
newval = []
for i in range (inv_data.shape[0]):
    listedval = list(inv_data.iloc[i].values)
    newval.append(listedval)
    
df.iloc[idx_inv] = newval
df.drop(['temp1'],1,inplace=True)

# Check dataframe after problem invalid value rows solved 
df.info()

# **Data Cleaning Process**

In [ ]:
# Check total null values in every column
pd.isna(df).sum()

In [ ]:
# Function to check and replace if there are '-' value in dataset.
def StripVal(col):
    newval = []
    for i in df[col].values:
        if str(i)=='—':
            newval.append(np.nan)
        else:
            newval.append(i)
    df[col]=newval

In [ ]:
# Apply StripVal function to every columns object
col_obj = df.select_dtypes('object').columns
for i in col_obj:
    StripVal(i)

In [ ]:
# Function to drop selected rows
def drop_val(col):
    idx_to_drop = []
    for i in col.index:
        idx_to_drop.append(i)

    df.drop(idx_to_drop,inplace=True)
    df.reset_index(drop=True,inplace=True)

In [ ]:
# If the car have no information about all of make, model, body, and trim columns so we cannot describe what car is it.

dataNull = df[(df['make'].isna()) & (df['model'].isna()) & (df['body'].isna()) & (df['trim'].isna())]

drop_val(dataNull)

In [ ]:
# If the car does not have color or interior information, dropped because it haven't got enough information.

data_no_color = df[(df['color'].isna()) | (df['interior'].isna())]

drop_val(data_no_color)

In [ ]:
# Change data type to number

df['year'] = df['year'].astype('int')
df['mmr'] = df['mmr'].astype('int')
df['sellingprice'] = df['sellingprice'].astype('int')
df['condition'] = df['condition'].astype('float')
df['odometer'] = df['odometer'].astype('float')

In [ ]:
#Fixing date columns

fixed_date = []
for i in range(len(df['saledate'])):
    date = df.saledate[i][:15]
    fixed_date.append(date)
    
df['saledate'] = fixed_date
df['saledate'] = pd.to_datetime(df['saledate'])

date_not_use = df[(df['saledate'] < '2014-12-01')]  # Deleting this data because there aren't enough data for that range of date 
drop_val(date_not_use)                              # It can cause trouble to EDA and Machine Learning modelling

In [ ]:
df.describe()

This table above show us statistical descriptive of the data, it give insight that some columns have unusual and potentially wrong data. For example:

* odometer have min and max value too far from the quartile, make the spread of data become ugly and have too large outlier
* mmr have minimal value so little that have to check again
* sellingprice have min and max value too far from the quartile like odometer column.

In [ ]:
# Fixing Sellingprice column

# Change column that have maximum sellingprice because its value is too big
# After some check, there is only one column that has maximum sellingprice value, that column have 22800 mmr
wr_val_sellingprice = df[df['sellingprice']==df['sellingprice'].max()].index

# Because the value of mmr and sellingprice is usually not too far, so I remove one '0' at this cell of sellingprice column
df.loc[wr_val_sellingprice,['sellingprice']] = 23000  

# Delete data that have sellingprice below 100, because it doesn't make sense
inv_sellingprice = df[df['sellingprice']<100]
drop_val(inv_sellingprice)